# Training Notebook

Train the CatMeowCNN model on preprocessed data.


In [1]:
import sys
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path("..").resolve()))
sys.path.insert(0, str(Path("../src").resolve()))

from src.data_loader import load_train_data, load_test_data, get_train_val_loaders, get_test_loader
from src.train import train, cross_validate
from src.test import test
from src.transforms import SpecAugment, Compose, RandomApply, AddNoise
from models import CatMeowCNN, TransferCNN
import matplotlib.pyplot as plt
import torch




## 1. Load Data


In [2]:
DATA_DIR = Path("../data/interim")
MODEL_PATH = Path("../results/cat_meow.pt")

# Ensure results dir exists
MODEL_PATH.parent.mkdir(parents=True, exist_ok=True)

# Load train and test separately
X_train, y_train = load_train_data(DATA_DIR)
X_test, y_test = load_test_data(DATA_DIR)

print(f"Train: X={X_train.shape}, y={y_train.shape}")
print(f"Test:  X={X_test.shape}, y={y_test.shape}")
print(f"Classes: {len(set(y_train))}")


Train: X=(80, 128, 173), y=(80,)
Test:  X=(20, 128, 173), y=(20,)
Classes: 10


In [3]:
# Augmentation for training
train_transform = Compose([
    SpecAugment(freq_mask_param=15, time_mask_param=25),
    RandomApply(AddNoise(noise_level=0.005), p=0.3),
])

# Split training data into train/val
train_loader, val_loader = get_train_val_loaders(
    X_train, y_train, batch_size=16, train_transform=train_transform
)
print(f"Train: {len(train_loader.dataset)} samples (with augmentation)")
print(f"Val: {len(val_loader.dataset)} samples")
print(f"Test: {len(X_test)} samples (held out)")


Train: 64 samples (with augmentation)
Val: 16 samples
Test: 20 samples (held out)


## 2. Create Model


In [4]:
n_classes = len(set(y_train))
model = CatMeowCNN(n_classes=n_classes)

n_params = sum(p.numel() for p in model.parameters())
print(f"Model: CatMeowCNN")
print(f"Parameters: {n_params:,}")


Model: CatMeowCNN
Parameters: 422,986


## 3. Train


In [ ]:
# Train a final model and evaluate on test set
model = CatMeowCNN(n_classes=n_classes)

# Get fresh loaders
train_loader, val_loader = get_train_val_loaders(
    X_train, y_train, batch_size=16, train_transform=train_transform
)

history = train(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    epochs=100,
    learning_rate=0.001,
    save_path=str(MODEL_PATH),
    patience=40,
)


## 5. Evaluate


In [ ]:
# Load best model and evaluate on held-out test data
model.load_state_dict(torch.load(MODEL_PATH, weights_only=True))
test_loader = get_test_loader(X_test, y_test)
results = test(model, test_loader)


## 6. Cross-Validation

Run k-fold cross-validation for a more reliable accuracy estimate with limited data.


In [ ]:
# Run 5-fold cross-validation on training data
cv_results = cross_validate(
    model_class=CatMeowCNN,
    X=X_train,
    y=y_train,
    n_splits=5,
    epochs=100,
    learning_rate=0.001,
    batch_size=16,
    patience=40,
    n_classes=n_classes,  # passed to CatMeowCNN
)


In [ ]:
# Visualize CV results
fig, ax = plt.subplots(figsize=(8, 4))
folds = [r["fold"] for r in cv_results["fold_results"]]
accs = cv_results["accuracies"]

ax.bar(folds, accs, color='steelblue', edgecolor='white')
ax.axhline(cv_results["mean_acc"], color='red', linestyle='--', 
           label=f'Mean: {cv_results["mean_acc"]:.3f} ± {cv_results["std_acc"]:.3f}')
ax.set_xlabel("Fold")
ax.set_ylabel("Validation Accuracy")
ax.set_title("5-Fold Cross-Validation Results")
ax.set_ylim(0, 1)
ax.legend()
plt.tight_layout()
plt.show()

print(f"\nCV Accuracy: {cv_results['mean_acc']:.1%} ± {cv_results['std_acc']:.1%}")


In [ ]:
# Plot training curves for each fold
n_folds = len(cv_results["fold_results"])
fig, axes = plt.subplots(n_folds, 2, figsize=(12, 3 * n_folds))

for i, fold_result in enumerate(cv_results["fold_results"]):
    fold_history = fold_result["history"]
    fold_num = fold_result["fold"]
    
    # Loss
    axes[i, 0].plot(fold_history["train_loss"], label="Train")
    axes[i, 0].plot(fold_history["val_loss"], label="Val")
    axes[i, 0].set_xlabel("Epoch")
    axes[i, 0].set_ylabel("Loss")
    axes[i, 0].set_title(f"Fold {fold_num} - Loss")
    axes[i, 0].legend()
    
    # Accuracy
    axes[i, 1].plot(fold_history["train_acc"], label="Train")
    axes[i, 1].plot(fold_history["val_acc"], label="Val")
    axes[i, 1].set_xlabel("Epoch")
    axes[i, 1].set_ylabel("Accuracy")
    axes[i, 1].set_title(f"Fold {fold_num} - Accuracy (Best: {fold_result['best_val_acc']:.3f})")
    axes[i, 1].legend()

plt.tight_layout()
plt.show()


## 8. Save Results for Diagnostics


In [ ]:
# Save all training results for diagnostics notebook
import pickle

RESULTS_DIR = Path("../results")
RESULTS_DIR.mkdir(parents=True, exist_ok=True)

# Bundle all results
training_results = {
    "history": history,
    "cv_results": cv_results,
    "test_results": results,
    "n_classes": n_classes,
}

with open(RESULTS_DIR / "training_results.pkl", "wb") as f:
    pickle.dump(training_results, f)

print(f"Saved training results to {RESULTS_DIR / 'training_results.pkl'}")
print(f"  - Training history: {len(history['train_loss'])} epochs")
print(f"  - CV results: {len(cv_results['fold_results'])} folds")
print(f"  - Test accuracy: {results['accuracy']:.1%}")


## 9. Model Comparison: CNN vs Transfer Learning

Compare our simple CatMeowCNN with transfer learning using pretrained ResNet.


In [ ]:
from models import TransferCNN

# Define models to compare
models_to_compare = {
    "CatMeowCNN": lambda: CatMeowCNN(n_classes=n_classes),
    "TransferCNN (frozen)": lambda: TransferCNN(n_classes=n_classes, backbone="resnet18", freeze_backbone=True),
    "TransferCNN (fine-tune)": lambda: TransferCNN(n_classes=n_classes, backbone="resnet18", freeze_backbone=False),
}

# Show model sizes
print("Model Comparison:")
print("-" * 50)
for name, model_fn in models_to_compare.items():
    m = model_fn()
    total = sum(p.numel() for p in m.parameters())
    trainable = sum(p.numel() for p in m.parameters() if p.requires_grad)
    print(f"{name:25} | Total: {total:>10,} | Trainable: {trainable:>10,}")


In [ ]:
# Run cross-validation for each model
comparison_results = {}

for name, model_fn in models_to_compare.items():
    print(f"\n{'='*60}")
    print(f"Training: {name}")
    print(f"{'='*60}")
    
    # Use lower learning rate for transfer learning
    lr = 0.0001 if "Transfer" in name else 0.001
    
    cv_result = cross_validate(
        model_class=model_fn,  # Pass the lambda function
        X=X_train,
        y=y_train,
        n_splits=5,
        epochs=50,  # Fewer epochs for comparison
        learning_rate=lr,
        batch_size=16,
        patience=15,
        n_classes=n_classes,
        verbose=False,  # Less output
    )
    
    comparison_results[name] = cv_result
    print(f"\n✓ {name}: {cv_result['mean_acc']:.1%} ± {cv_result['std_acc']:.1%}")


In [ ]:
# Visualize comparison results
import numpy as np

fig, ax = plt.subplots(figsize=(10, 6))

model_names = list(comparison_results.keys())
means = [comparison_results[name]["mean_acc"] for name in model_names]
stds = [comparison_results[name]["std_acc"] for name in model_names]

x = np.arange(len(model_names))
colors = ['#3498db', '#2ecc71', '#e74c3c']

bars = ax.bar(x, means, yerr=stds, capsize=10, color=colors, 
              edgecolor='white', linewidth=2, error_kw={'linewidth': 2})

ax.set_xlabel("Model", fontsize=12)
ax.set_ylabel("CV Accuracy", fontsize=12)
ax.set_title("Model Comparison (5-Fold Cross-Validation)", fontsize=14)
ax.set_xticks(x)
ax.set_xticklabels(model_names, rotation=15, ha='right')
ax.set_ylim(0, 1)
ax.axhline(0.5, color='gray', linestyle='--', alpha=0.5, label='Random baseline')
ax.grid(True, alpha=0.3, axis='y')

# Add value labels
for bar, mean, std in zip(bars, means, stds):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + std + 0.03,
            f'{mean:.1%}', ha='center', va='bottom', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

# Summary table
print("\n" + "="*60)
print("SUMMARY")
print("="*60)
for name in model_names:
    r = comparison_results[name]
    print(f"{name:25} | {r['mean_acc']:.1%} ± {r['std_acc']:.1%}")
print("="*60)

# Find best model
best_model = max(comparison_results.keys(), key=lambda k: comparison_results[k]['mean_acc'])
print(f"\n🏆 Best Model: {best_model}")


### Train Best Model on Full Data


In [ ]:
# Train the best model on full training data
BEST_MODEL_PATH = Path("../results/best_model.pt")

# Use TransferCNN with frozen backbone (usually best for small data)
best_model = TransferCNN(n_classes=n_classes, backbone="resnet18", freeze_backbone=True)

# Get fresh loaders
train_loader, val_loader = get_train_val_loaders(
    X_train, y_train, batch_size=16, train_transform=train_transform
)

print(f"Training best model: TransferCNN (frozen ResNet18)")
print(f"Trainable params: {sum(p.numel() for p in best_model.parameters() if p.requires_grad):,}")

best_history = train(
    model=best_model,
    train_loader=train_loader,
    val_loader=val_loader,
    epochs=100,
    learning_rate=0.0001,  # Lower LR for transfer learning
    save_path=str(BEST_MODEL_PATH),
    patience=20,
)


In [ ]:
# Evaluate best model on test set
best_model.load_state_dict(torch.load(BEST_MODEL_PATH, weights_only=True))
test_loader = get_test_loader(X_test, y_test)
best_results = test(best_model, test_loader)

print("\n" + "="*50)
print("FINAL COMPARISON")
print("="*50)
print(f"CatMeowCNN Test Accuracy:    {results['accuracy']:.1%}")
print(f"TransferCNN Test Accuracy:   {best_results['accuracy']:.1%}")
print(f"Improvement:                 {(best_results['accuracy'] - results['accuracy'])*100:+.1f} percentage points")
print("="*50)


## 10. Hyperparameter Optimization (Bayesian)

Using Optuna with TPE sampler to find optimal hyperparameters.

TPE: https://arxiv.org/abs/2304.11127


In [ ]:
import optuna
from optuna.samplers import TPESampler

# Suppress Optuna logs (optional)
optuna.logging.set_verbosity(optuna.logging.WARNING)

def objective(trial):
    """Objective function for Optuna optimization."""
    
    # Hyperparameters to tune
    model_type = trial.suggest_categorical("model", ["CatMeowCNN", "TransferCNN"])
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 16])
    
    # Optimizer and regularization
    optimizer_type = trial.suggest_categorical("optimizer", ["adam", "sgd"])
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-2, log=True)
    dropout = trial.suggest_float("dropout", 0.3, 0.7)
    
    # Model-specific params
    if model_type == "TransferCNN":
        freeze = trial.suggest_categorical("freeze_backbone", [True, False])
        backbone = trial.suggest_categorical("backbone", ["resnet18", "resnet34"])
        model_fn = lambda: TransferCNN(
            n_classes=n_classes, 
            backbone=backbone, 
            freeze_backbone=freeze,
            dropout=dropout
        )
    else:
        model_fn = lambda: CatMeowCNN(n_classes=n_classes, dropout=dropout)
    
    # Quick 3-fold CV (faster than 5-fold for tuning)
    try:
        cv_result = cross_validate(
            model_class=model_fn,
            X=X_train,
            y=y_train,
            n_splits=3,
            epochs=100,  # Full training
            learning_rate=lr,
            batch_size=batch_size,
            patience=30,  # More patience for convergence
            n_classes=n_classes,
            verbose=False,
            optimizer_type=optimizer_type,
            weight_decay=weight_decay,
        )
        return cv_result["mean_acc"]
    except Exception as e:
        print(f"Trial failed: {e}")
        return 0.0

print("Objective function defined. Ready to optimize!")
print("Search space: model, lr, batch_size, optimizer, weight_decay, dropout")


Objective function defined. Ready to optimize!
Search space: model, lr, batch_size, optimizer, weight_decay, dropout


/Users/bugrasipahioglu/repo/cat-sound-classifier/venv/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# Run Bayesian optimization

N_TRIALS = 1  # Reduce for faster testing, increase for better results

print(f"Starting Bayesian Optimization with {N_TRIALS} trials...")

study = optuna.create_study(
    direction="maximize",  # Maximize accuracy
    sampler=TPESampler(seed=42),
    study_name="cat_mood_classifier"
)

study.optimize(
    objective, 
    n_trials=N_TRIALS, 
    show_progress_bar=True,
    callbacks=[lambda study, trial: print(f"  Trial {trial.number}: {trial.value:.1%} | {trial.params}")]
)

print("\n" + "="*60)
print("🎉 OPTIMIZATION COMPLETE!")
print("="*60)


Starting Bayesian Optimization with 1 trials...


Best trial: 0. Best value: 0.311016: 100%|██████████| 1/1 [00:17<00:00, 17.68s/it]

  Trial 0: 31.1% | {'model': 'TransferCNN', 'lr': 0.001570297088405539, 'batch_size': 8, 'optimizer': 'sgd', 'weight_decay': 0.00025378155082656634, 'dropout': 0.5832290311184182, 'freeze_backbone': False, 'backbone': 'resnet18'}

🎉 OPTIMIZATION COMPLETE!


In [ ]:
# Display best results
print(f"Best CV Accuracy: {study.best_value:.1%}")
print(f"\nBest Hyperparameters:")
for param, value in study.best_params.items():
    print(f"  {param}: {value}")

# Show top 5 trials
print("\nTop 5 Trials:")
print("-" * 60)
trials_df = study.trials_dataframe().sort_values("value", ascending=False).head(5)
for i, row in trials_df.iterrows():
    print(f"  {row['value']:.1%} | model={row.get('params_model', 'N/A')}, lr={row.get('params_lr', 0):.2e}, batch={row.get('params_batch_size', 'N/A')}")


In [ ]:
# Visualize optimization history
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Optimization history
trials = [t.number for t in study.trials]
values = [t.value for t in study.trials]
best_so_far = [max(values[:i+1]) for i in range(len(values))]

axes[0].scatter(trials, values, alpha=0.6, label="Trial accuracy")
axes[0].plot(trials, best_so_far, 'r-', linewidth=2, label="Best so far")
axes[0].set_xlabel("Trial")
axes[0].set_ylabel("CV Accuracy")
axes[0].set_title("Optimization History")
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Right: Parameter importance (simplified)
param_counts = {}
for t in study.trials:
    if t.value >= study.best_value * 0.95:  # Top 5% trials
        for k, v in t.params.items():
            key = f"{k}={v}"
            param_counts[key] = param_counts.get(key, 0) + 1

if param_counts:
    sorted_params = sorted(param_counts.items(), key=lambda x: -x[1])[:10]
    params, counts = zip(*sorted_params)
    axes[1].barh(range(len(params)), counts, color='steelblue')
    axes[1].set_yticks(range(len(params)))
    axes[1].set_yticklabels(params)
    axes[1].set_xlabel("Count in top trials")
    axes[1].set_title("Common Parameters in Best Trials")

plt.tight_layout()
plt.show()


### Train Final Model with Best Hyperparameters


In [ ]:
# Train final model with best hyperparameters
TUNED_MODEL_PATH = Path("../results/tuned_model.pt")

best = study.best_params
print(f"Training final model with best params:")
for k, v in best.items():
    if isinstance(v, float):
        print(f"  {k}: {v:.6f}")
    else:
        print(f"  {k}: {v}")

# Create model based on best params
if best["model"] == "TransferCNN":
    final_model = TransferCNN(
        n_classes=n_classes, 
        backbone=best.get("backbone", "resnet18"),
        freeze_backbone=best.get("freeze_backbone", True),
        dropout=best.get("dropout", 0.5)
    )
else:
    final_model = CatMeowCNN(n_classes=n_classes, dropout=best.get("dropout", 0.5))

trainable = sum(p.numel() for p in final_model.parameters() if p.requires_grad)
print(f"\nModel: {best['model']}")
print(f"Trainable params: {trainable:,}")

# Get fresh loaders with best batch size
train_loader, val_loader = get_train_val_loaders(
    X_train, y_train, 
    batch_size=best["batch_size"], 
    train_transform=train_transform
)

# Train with more epochs now, using best optimizer and regularization
tuned_history = train(
    model=final_model,
    train_loader=train_loader,
    val_loader=val_loader,
    epochs=100,
    learning_rate=best["lr"],
    save_path=str(TUNED_MODEL_PATH),
    patience=50,  # Match tuning patience
    optimizer_type=best.get("optimizer", "adam"),
    weight_decay=best.get("weight_decay", 0.0),
)


In [ ]:
# Final evaluation on test set
final_model.load_state_dict(torch.load(TUNED_MODEL_PATH, weights_only=True))
test_loader = get_test_loader(X_test, y_test)
tuned_results = test(final_model, test_loader)

print("\n" + "="*60)
print("🏆 FINAL RESULTS")
print("="*60)
print(f"Best CV Accuracy (during tuning): {study.best_value:.1%}")
print(f"Test Accuracy (tuned model):      {tuned_results['accuracy']:.1%}")
print(f"Test Loss:                        {tuned_results['loss']:.4f}")
print("="*60)
print(f"\nBest hyperparameters saved. Model saved to: {TUNED_MODEL_PATH}")


In [ ]:
# Save tuning results for diagnostics notebook
tuning_results = {
    "tuned_history": tuned_history,
    "tuned_results": tuned_results,
    "best_params": study.best_params,
    "best_cv_acc": study.best_value,
    "n_classes": n_classes,
    "optimization_history": [
        {"trial": t.number, "value": t.value, "params": t.params}
        for t in study.trials if t.value is not None
    ],
}

with open(RESULTS_DIR / "tuning_results.pkl", "wb") as f:
    pickle.dump(tuning_results, f)

print(f"Saved tuning results to {RESULTS_DIR / 'tuning_results.pkl'}")
print(f"  - Training history: {len(tuned_history['train_loss'])} epochs")
print(f"  - Best CV accuracy: {study.best_value:.1%}")
print(f"  - Test accuracy: {tuned_results['accuracy']:.1%}")
